# Preparing the pickle files for `State-evolution` cluster runs of particles transitioning through the quadrupole lens

Particles propagate from a homogenous field in z to the quadrupole field in the yz plane back into a homogeneous field in z.  
The homogeneous field can be scaled with a scaling factor, where `scaling factor = 1` corresponds to 84V/cm.
The `scaling factor` and `trajectory idx` are iterated over on the cluster

In [1]:
from utils.fields import *
from utils.trajectories import *
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
pickle_dir = 'options pickles/'
trajectory_dir = 'trajectories/'

In [3]:
from trajectories import *
positions = load_trajectories(trajectory_dir+'positions_545.mat')
velocities = load_velocities(trajectory_dir+'velocities_545.mat')

t, (xT, yT, zT), (xVelocity, yVelocity, zVelocity) = get_time_position_velocity(positions, velocities)

In [4]:
def makeInterpolate(t,x,y,z):
    """
    Note that for the field simulations the molecular
    beam directin is z.
    Interpolating the trajectories, quadruatic for 
    gravitational shift, linear otherwise.
    """
    intTrajX = interp1d(t, x, kind = 'linear')
    intTrajY = interp1d(t, y, kind = 'quadratic')
    intTrajZ = interp1d(t, z, kind = 'linear')

    rootfun = lambda t: intTrajZ(t) - (0.8052-0.15)
    tstart = brentq(rootfun, 0.001, 0.015)
    rootfun = lambda t: intTrajZ(t) - (0.8052+0.6+0.15)
    tstop = brentq(rootfun, 0.001, 0.015)
    
    intTrajX = interp1d(t-tstart, x, kind = 'linear')
    intTrajY = interp1d(t-tstart, y, kind = 'quadratic')
    intTrajZ = interp1d(t-tstart, z-(0.8052-0.15)-0.45, kind = 'linear')
    
    return intTrajX, intTrajY, intTrajZ, tstop-tstart

In [5]:
interpolateTrajectories = []
for idInt in range(t.shape[1]):
    interpolateTrajectories.append((makeInterpolate(t[:,idInt], xT[:,idInt], yT[:,idInt], zT[:,idInt])))

In [6]:
del t, xT, yT, zT, xVelocity, yVelocity, zVelocity

generating the interpolating functions for the homogeneous potential and quadrupole potential  
`quadrupole field` : `c` is a matrix containing the 3rd order polynomial terms in x and y; `intfun` is a spline function for the field along the z axis  
`homogeneous field` : `cP` is a matrix containing the 3rd order polynomial terms in x and y; `intfunP` is a spline function for the field along the z axis

In [7]:
c, intfun = fit_potential_xyz("D:/Adiabatic Transfer/propagation/fields/"+'potentialQuadrupole.fld', 0.022, 0.019, 0, 0)
cP, intfunP = fit_potential_xyz('D:/Adiabatic Transfer/propagation/fields/'+'potentialElectrodes.fld', 0.022, 0.019, .45, 0)

taking the derivatives of the polynomial fits to facilitate the field calculations in `State-evolution`

In [8]:
from numpy.polynomial import polynomial

x_cder = polynomial.polyder(c, axis = 0)
x_cPder = polynomial.polyder(cP, axis = 0)
y_cder = polynomial.polyder(c, axis = 1)
y_cPder = polynomial.polyder(cP, axis = 1)

pickling the spline functions and 2d polynomial matrices for use in `State-evolution`

In [9]:
import pickle 

with open(pickle_dir+'potential_plate_z_run1.pkl', 'wb') as wfp:
    pickle.dump(intfunP, wfp)
    
with open(pickle_dir+'potential_quadrupole_z_run1.pkl', 'wb') as wfp:
    pickle.dump(intfun, wfp)
    
with open(pickle_dir+'potential_plate_xy_run1.pkl', 'wb') as wfp:
    pickle.dump([cP, x_cPder, y_cPder], wfp)
    
with open(pickle_dir+'potential_quadrupole_xy_run1.pkl', 'wb') as wfp:
    pickle.dump([c, x_cder, y_cder], wfp)
    
with open(pickle_dir+'trajectories_run1.pkl', 'wb') as wfp:
    pickle.dump(interpolateTrajectories, wfp)